# Semantic Kernel入門

このコードサンプルでは、[Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AIフレームワークを使用して基本的なエージェントを作成します。

このサンプルの目標は、異なるAgenticパターンを実装する際に、後の追加コードサンプルで使用する手順を示すことです。

## 📚 このノートブックで学習できること

### 🎯 主要な学習目標
- **Semantic Kernel**の基本的な使用方法
- **GitHub Models**との連携によるLLMアクセス
- **プラグインシステム**による機能拡張
- **基本的なAIエージェント**の実装

### 🛠️ 実装する技術スタック
- **Semantic Kernel**: Microsoft製AIエージェントフレームワーク
- **GitHub Models**: gpt-4o-miniモデルへのアクセス
- **プラグイン開発**: カスタム機能の実装
- **非同期処理**: ストリーミング応答の実装

### 💡 実用的なスキル
- AIエージェントの基本構造理解
- プラグインによる機能拡張方法
- 旅行エージェントの実用的実装例

## 必要なPythonパッケージのインポート

In [1]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`ai_model_id`は`gpt-4o-mini`として定義されています。GitHub Modelsマーケットプレイスで利用可能な他のモデルに変更して、異なる結果を確認してみてください。

GitHub ModelsのベースURLに使用される`Azure Inference SDK`を使用するために、Semantic Kernel内の`OpenAIChatCompletion`コネクタを使用します。他のモデルプロバイダーでSemantic Kernelを使用するための[他の利用可能なコネクタ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)もあります。

In [2]:
import random   

# サンプル用のプラグインを定義

class DestinationsPlugin:
    """バケーション用のランダムな目的地リスト。"""

    def __init__(self):
        # バケーション目的地のリスト
        self.destinations = [
            "バルセロナ、スペイン",
            "パリ、フランス",
            "ベルリン、ドイツ",
            "東京、日本",
            "シドニー、オーストラリア",
            "ニューヨーク、アメリカ",
            "カイロ、エジプト",
            "ケープタウン、南アフリカ",
            "リオデジャネイロ、ブラジル",
            "バリ、インドネシア"
        ]
        # 重複を避けるために最後の目的地を記録
        self.last_destination = None

    @kernel_function(description="ランダムなバケーション目的地を提供します。")
    def get_random_destination(self) -> Annotated[str, "ランダムなバケーション目的地を返します。"]:
        # 利用可能な目的地を取得（可能であれば最後の目的地を除外）
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # ランダムな目的地を選択
        destination = random.choice(available_destinations)

        # 最後の目的地を更新
        self.last_destination = destination

        return destination

In [3]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# `ChatCompletionAgent`で使用されるAIサービスを作成
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## エージェントの作成

以下では`TravelAgent`というエージェントを作成します。

この例では、非常にシンプルな指示を使用しています。これらの指示を変更して、エージェントがどのように異なって応答するかを確認してみてください。

In [4]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="あなたはランダムな目的地で顧客のバケーション計画を支援できる有用なAIエージェントです",
)

## エージェントの実行

`ChatHistoryAgentThread`タイプのスレッドを定義することで、エージェントを実行できます。必要なシステムメッセージは、エージェントのinvoke_streamの`messages`キーワード引数に提供されます。

これらが定義された後、ユーザーがエージェントに送信する内容となる`user_inputs`を作成します。この場合、このメッセージを「日帰り旅行を計画してください」に設定しています。

エージェントがどのように異なって応答するかを確認するために、このメッセージを自由に変更してみてください。

In [5]:
async def main():
    # エージェント用の新しいスレッドを作成
    # スレッドが提供されない場合、新しいスレッドが
    # 作成され、初期応答と共に返されます
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "日帰り旅行を計画してください。",
    ]

    for user_input in user_inputs:
        print(f"# ユーザー: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. 応答を出力
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # スレッドをクリーンアップ
    await thread.delete() if thread else None

await main()

# ユーザー: 日帰り旅行を計画してください。

# TravelAgent: # TravelAgent: 日帰り旅行の目的地として日帰り旅行の目的地として、南アフリカのケ、南アフリカのケープタウンを提ープタウンを提案します！ 

###案します！ 

### ケープタウンの日帰 ケープタウンの日帰り旅行プランり旅行プラン

**午前:**
- **

**午前:**
- **テーブルマウンテン**:テーブルマウンテン**: ケーブルカーに乗 ケーブルカーに乗って、絶景のパって、絶景のパノラマビューを楽しノラマビューを楽しみましょう。
みましょう。
- **朝食**: ケ- **朝食**: ケーブルカーの近くにーブルカーの近くにあるカフェで地あるカフェで地元の料理を楽しむ元の料理を楽しむのもいいのもいいでしょう。

**午後:でしょう。

**午後:**
- **ブ**
- **ブランズウィランズウィックビーチックビーチ**: 美しい**: 美しい白砂のビ白砂のビーチでリラックスーチでリラックスや散歩を楽しんや散歩を楽しんでみてください。
でみてください。
- **ヴィクトリア- **ヴィクトリア＆アルフレッド・ウォ＆アルフレッド・ウォーーターフターフロント**: お土ロント**: お土産店やレストラン産店やレストランがある、活気あるエリアがある、活気あるエリアを散策しますを散策します。

**夕方:**
- **夕。

**夕方:**
- **夕食**: 海の景色を堪食**: 海の景色を堪能できるレ能できるレストランで新ストランで新鮮なシーフ鮮なシーフードを楽しむードを楽しむのはいかがでしょうか。

**のはいかがでしょうか。

**夜:**
- **カ夜:**
- **カフカの夕日**: ケープフカの夕日**: ケープタウンの素晴らしいタウンの素晴らしい夕日を見ながら、一夕日を見ながら、一日を締めくくり日を締めくくりましょう。

ケープタウンはましょう。

ケープタウンは文化、美食、自然が融合した文化、美食、自然が融合した素敵な場所です素敵な場所です。。素晴らしい日素晴らしい日帰り旅行帰り旅行になりますように！になりますように！

